In [21]:
from google.cloud import bigquery
from bs4 import BeautifulSoup
import os

In [22]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/Users/jason/Documents/Jason/JobMatch/job-match-271401-74d3c9eb9112.json'

In [23]:
query = """
select *
from job_search.job_details
where search_datetime between '2020-05-01' and '2020-05-03'
and lower(title) like '%data%sci%'
"""

In [24]:
bq_client = bigquery.Client()
df = bq_client.query(query).result().to_dataframe()

In [25]:
df.shape

(62, 11)

In [26]:
df.head()

,job_id,search_datetime,location,search_title,search_location,search_detail_datetime,title,job_desc,company,company_rating,company_rating_count
0,44581b261abec239,2020-05-02 15:29:00+00:00,"Dallas, TX",IT+Engineer,Dallas,2020-05-04 02:01:00+00:00,"Sr. Data Scientist (Dallas, TX)","<div class=""jobsearch-jobDescriptionText"" id=""...",Tiger Analytics,0.0,0
1,16bdb1763913307c,2020-05-02 23:14:00+00:00,"Louisville, CO 80027",data+analyst,Denver,2020-05-04 02:01:00+00:00,Data Science Internship,"<div class=""jobsearch-jobDescriptionText"" id=""...",GHX,3.2,22
2,8dc961b84c2d9b33,2020-05-02 23:14:00+00:00,"Tampa, FL 33607",data+analyst,Tampa,2020-05-04 02:00:00+00:00,Director of Data Science & Analytics,"<div class=""jobsearch-jobDescriptionText"" id=""...",ClearSource,3.8,69
3,60baeac4d4fbd010,2020-05-02 23:12:00+00:00,"Omaha, NE",data+scientist,Omaha,2020-05-04 02:00:00+00:00,Data Scientist,"<div class=""jobsearch-jobDescriptionText"" id=""...","GrainBridge, LLC",0.0,0
4,fb253e56a646292a,2020-05-02 23:13:00+00:00,"Reston, VA",data+scientist,Reston,2020-05-04 02:01:00+00:00,Senior Data Science Consultant,"<div class=""jobsearch-jobDescriptionText"" id=""...",Raft,0.0,0


In [27]:
df['job_desc'] = df['job_desc'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)

In [28]:
df.head()

,job_id,search_datetime,location,search_title,search_location,search_detail_datetime,title,job_desc,company,company_rating,company_rating_count
0,44581b261abec239,2020-05-02 15:29:00+00:00,"Dallas, TX",IT+Engineer,Dallas,2020-05-04 02:01:00+00:00,"Sr. Data Scientist (Dallas, TX)",Tiger Analytics is looking for experienced Dat...,Tiger Analytics,0.0,0
1,16bdb1763913307c,2020-05-02 23:14:00+00:00,"Louisville, CO 80027",data+analyst,Denver,2020-05-04 02:01:00+00:00,Data Science Internship,The Intern would work in support of Data Scien...,GHX,3.2,22
2,8dc961b84c2d9b33,2020-05-02 23:14:00+00:00,"Tampa, FL 33607",data+analyst,Tampa,2020-05-04 02:00:00+00:00,Director of Data Science & Analytics,As the Director of the eCommerce Data Science ...,ClearSource,3.8,69
3,60baeac4d4fbd010,2020-05-02 23:12:00+00:00,"Omaha, NE",data+scientist,Omaha,2020-05-04 02:00:00+00:00,Data Scientist,DESCRIPTION\nGrainBridge is seeking a talented...,"GrainBridge, LLC",0.0,0
4,fb253e56a646292a,2020-05-02 23:13:00+00:00,"Reston, VA",data+scientist,Reston,2020-05-04 02:01:00+00:00,Senior Data Science Consultant,Who we are: We (http://goraft.tech/) are growi...,Raft,0.0,0


In [61]:
df.job_desc[21]

"Chase:\n\nChase is the U.S. consumer and commercial banking business of JPMorgan Chase & Co. (NYSE: JPM), a leading global financial services firm with assets of $2.5 trillion and operations worldwide. Chase serves nearly half of America's households with a broad range of financial services, including personal banking, credit cards, mortgages, auto financing, investment advice, small business loans and payment processing. Customers can choose how and where they want to bank: 5,200 branches, 16,000 ATMs, mobile, online and by phone. For more information, go to Chase.com.\n\nChase Consumer & Community Bank (CCB) serves 61 million households and 4 million small businesses with a broad range of financial services, including banking, credit cards, auto finance, payments, mortgages, and investment advice.\n\n\nCCB Data & Analytics:\n\nCCB Data & Analytics Team unifies data and analytics talent across Chase to responsibly leverage data to build competitive advantages for our businesses with 

# NLP Spacy

In [42]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')

In [51]:
df['job_desc_POS'] = df['job_desc'].apply(lambda x: nlp(x))

In [52]:
doc = df.job_desc_POS[9]

In [53]:
for token in doc:
    print(f'{token.text:{10}} {token.pos_:{8}} {token.tag_:{6}} {spacy.explain(token.tag_)}')

Overview   NOUN     NN     noun, singular or mass
:          PUNCT    :      punctuation mark, colon or ellipsis

          SPACE           None
Interested PROPN    NNP    noun, proper singular
in         ADP      IN     conjunction, subordinating or preposition
working    VERB     VBG    verb, gerund or present participle
with       ADP      IN     conjunction, subordinating or preposition
talented   ADJ      JJ     adjective
people     NOUN     NNS    noun, plural
to         PART     TO     infinitival to
help       VERB     VB     verb, base form
develop    VERB     VB     verb, base form
innovative ADJ      JJ     adjective
solutions  NOUN     NNS    noun, plural
to         ADP      IN     conjunction, subordinating or preposition
some       DET      DT     determiner
of         ADP      IN     conjunction, subordinating or preposition
society    NOUN     NN     noun, singular or mass
’s         PART     POS    possessive ending
most       ADV      RBS    adverb, superlative
comple

In [54]:
print(doc)

Overview:
Interested in working with talented people to help develop innovative solutions to some of society’s most complex and challenging problems? We are Guidehouse, a leading consulting firm serving the public sector and commercial clients with specialized capabilities in strategy, technology, and risk management. You may not yet know our name, but we have a rich history. Guidehouse is a combination of PwC’s former public sector practice and Navigant’s deep expertise in energy, financial services and healthcare.

We offer an exciting, fast-paced environment that fosters intellectual growth and rewards individuals based on impact, not tenure. Our firm is at the forefront of an emerging model solving complex problems that stretch across government and private companies, affording our people the opportunity to be on the cutting edge of the consulting profession. By focusing on markets facing transformational change, technology-driven innovation, and significant regulatory pressure, ou

In [ ]:
displacy.render(doc, style='ent', jupyter=True, options={'distance': 110})

In [43]:
query_tag = """
select tag_name
from `bigquery-public-data.stackoverflow.tags` 
order by count desc
limit 1000;
"""

In [44]:
bq_client = bigquery.Client()
df_tag = bq_client.query(query_tag).result().to_dataframe()

In [46]:
tag_list = df_tag.tag_name.tolist()

In [48]:
from spacy.matcher import PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)

In [49]:
patterns = list(nlp.pipe(tag_list))

In [50]:
# Apply the patterns to our matcher object:
matcher.add('tech_skills', None, *patterns)



In [55]:
# Apply the matcher to our Doc object:
matches = matcher(doc)

# See what matches occur:
matches

[(9008955172966653837, 124, 125),
 (9008955172966653837, 177, 178),
 (9008955172966653837, 265, 266),
 (9008955172966653837, 328, 329),
 (9008955172966653837, 399, 400),
 (9008955172966653837, 414, 415),
 (9008955172966653837, 526, 527),
 (9008955172966653837, 554, 555),
 (9008955172966653837, 678, 679),
 (9008955172966653837, 684, 685),
 (9008955172966653837, 871, 872),
 (9008955172966653837, 893, 894),
 (9008955172966653837, 930, 931),
 (9008955172966653837, 970, 971),
 (9008955172966653837, 979, 980),
 (9008955172966653837, 994, 995)]

In [56]:
for match_id, start, end in matches:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

9008955172966653837 tech_skills 124 125 model
9008955172966653837 tech_skills 177 178 class
9008955172966653837 tech_skills 265 266 client
9008955172966653837 tech_skills 328 329 image
9008955172966653837 tech_skills 399 400 testing
9008955172966653837 tech_skills 414 415 cloud
9008955172966653837 tech_skills 526 527 model
9008955172966653837 tech_skills 554 555 matplotlib
9008955172966653837 tech_skills 678 679 position
9008955172966653837 tech_skills 684 685 background
9008955172966653837 tech_skills 871 872 position
9008955172966653837 tech_skills 893 894 email
9008955172966653837 tech_skills 930 931 search
9008955172966653837 tech_skills 970 971 package
9008955172966653837 tech_skills 979 980 package
9008955172966653837 tech_skills 994 995 include


In [57]:
# Write a function to display basic entity info:
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))
    else:
        print('No named entities found.')

In [58]:
# Here we create Spans from each match, and create named entities from them:
from spacy.tokens import Span

PROD = doc.vocab.strings[u'TECH SKILLS']

new_ents = [Span(doc, match[1],match[2],label=PROD) for match in matches]

doc.ents = list(doc.ents) + new_ents

ValueError: [E084] Error assigning label ID 13978071407448052641 to span: not in StringStore.

In [59]:
show_ents(doc)

Guidehouse - ORG - Companies, agencies, institutions, etc.
Guidehouse - ORG - Companies, agencies, institutions, etc.
PwC’s - ORG - Companies, agencies, institutions, etc.
Navigant - PERSON - People, including fictional


 - CARDINAL - Numerals that do not fall under another type
Guidehouse - ORG - Companies, agencies, institutions, etc.
Advanced Data Analytics - ORG - Companies, agencies, institutions, etc.
Guidehouse - ORG - Companies, agencies, institutions, etc.


 - CARDINAL - Numerals that do not fall under another type
Data Science - ORG - Companies, agencies, institutions, etc.
the Artificial Intelligence Center of Excellence - ORG - Companies, agencies, institutions, etc.
CoE - ORG - Companies, agencies, institutions, etc.
CoE - ORG - Companies, agencies, institutions, etc.
Guidehouse - ORG - Companies, agencies, institutions, etc.
Advanced Analytics - ORG - Companies, agencies, institutions, etc.
CoE - ORG - Companies, agencies, institutions, etc.
NLP - ORG - Companies, agenc

In [ ]:
# https://stackoverflow.com/questions/47638877/using-phrasematcher-in-spacy-to-find-multiple-match-types
# https://course.spacy.io/en/chapter2

In [ ]:
tech_skills = ['Python', 'ReactJS', 'MongoDB','SQL','JavaScript', 'Tableau', 'Java'
               'R', 'Matlab', 'Julia', 'Matlab', 'Mathematica', 'Spark', 'PySpark' ,'Hadoop', 'NoSQL', 'Postgres',
              'S3', 'EC2', 'MapReduce', 'Pig', 'Hive', 'HTML5', 'CSS', 'R Shiny', 'Power BI', 'matplotlib', 
               'seaborn', 'bokeh', 'pandas', 'JIRA', 'Agile', 'Natural Language Processing',
              'AWS', 'Azure', 'Google Cloud', 'GCP', 'Databricks', 'Deep Learning', 'Graph database', 'Scala',
               'SAS','C++', 'Linux','Spotfire', 'Oracle','Apache', 'SPSS','SAP', 'Teradata'
              ]
concepts = ['machine learning', 'Business Intelligence', 'Computer Science', 'Statistics', 'Economics',
            'Physics', 'Engineering', 'Mathematics','feature engineering', 'recommender systems', 'NLP', 
            'computer vision', 'Information Systems', 'Economics', 'ML', 'AI'
           ]
